In [3]:
df_bridge = pd.read_excel(r'C:\Users\bjama\Desktop\ASIM\assignment 1\WBSIM_Lab1_2024\infrastructure\BMMS_overview.xlsx')
df_bridge.head()


,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
0,N1,1.800,Box Culvert,LRP001a,.,11.30,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.8,19.5,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702889,90.450389,bcs1
1,N1,4.925,Box Culvert,LRP004b,.,6.60,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.4,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.693611,90.478833,bcs1
2,N1,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.704583,90.518833,road_precise
3,N1,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.30,A,112531,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.88,12.2,1992.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699833,90.530722,bcs1
4,N1,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.30,A,112532,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.897,12.2,1984.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699667,90.530722,bcs1


# Check LRPs that swapped coordinates and are out of the Bangladesh polygon

In [5]:
import geopandas as gpd
from shapely.geometry import Point

# Load boundary polygon
bgd = gpd.read_file(r'C:\Users\bjama\Desktop\ASIM\ASIM_Team14\EPA133a-G14-A1\data\gis\gadm\BGD_adm0.shp')
boundary = bgd.union_all()

# Create geometries for original and swapped coordinates
geom_original = df_bridge.apply(lambda r: Point(r['lon'], r['lat']) if pd.notna(r['lon']) and pd.notna(r['lat']) else None, axis=1)
geom_swapped = df_bridge.apply(lambda r: Point(r['lat'], r['lon']) if pd.notna(r['lon']) and pd.notna(r['lat']) else None, axis=1)

# Flag rows where original is outside but swapped is inside
df_bridge['in_boundary'] = geom_original.apply(lambda p: p.within(boundary) if p else False)
df_bridge['in_boundary_swapped'] = geom_swapped.apply(lambda p: p.within(boundary) if p else False)
df_bridge['lat_lon_swapped_flag'] = (~df_bridge['in_boundary']) & (df_bridge['in_boundary_swapped'])

# Show flagged rows
df_bridge[df_bridge['lat_lon_swapped_flag']]

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,...,zone,circle,division,sub-division,lat,lon,EstimatedLoc,in_boundary,in_boundary_swapped,lat_lon_swapped_flag
2864,R241,29.840,Box Culvert,LRP029a,MAJIDEPUR BOX CULVERT,3.0,A,114991,Auskandi-Raniganj-Jagannathpur-Pagla Road,29.84,...,Sylhet,Sylhet,Sunamganj,Chattak,91.543889,24.805111,bcs1,False,True,True
2865,R241,31.212,Baily with Steel Deck,LRP031a,KHASIRA BAILEY WITH STEEL DECK,17.5,A,114990,Auskandi-Raniganj-Jagannathpur-Pagla Road,31.212,...,Sylhet,Sylhet,Sunamganj,Chattak,91.538722,24.816556,bcs1,False,True,True
2866,R241,32.713,Box Culvert,LRP032b,KOLKALI BOX CULVERT,3.0,A,114988,Auskandi-Raniganj-Jagannathpur-Pagla Road,32.713,...,Sylhet,Sylhet,Sunamganj,Chattak,91.530556,24.828056,bcs1,False,True,True
2867,R241,33.126,Box Culvert,LRP033a,KOKAL BOX CULVERT,1.0,A,114987,Auskandi-Raniganj-Jagannathpur-Pagla Road,33.126,...,Sylhet,Sylhet,Sunamganj,Chattak,91.527917,24.830944,bcs1,False,True,True
2868,R241,34.978,Box Culvert,LRP035a,TARAKHAI BOX CULVERT,14.4,A,114984,Auskandi-Raniganj-Jagannathpur-Pagla Road,34.978,...,Sylhet,Sylhet,Sunamganj,Chattak,91.518194,24.844972,bcs1,False,True,True
2869,R241,36.164,Truss with Steel Deck,LRP036a,COUNDER NOLA BRIDGE,40.6,A,114982,Auskandi-Raniganj-Jagannathpur-Pagla Road,36.164,...,Sylhet,Sylhet,Sunamganj,Chattak,91.514083,24.855083,bcs1,False,True,True
16183,R241,26.292,Box Culvert,LRP026a,EKOR CHOL BOX CULVERT,12.0,B,114995,Auskandi-Raniganj-Jagannathpur-Pagla Road,26.292,...,Sylhet,Sylhet,Sunamganj,Chattak,91.544194,24.773694,bcs1,False,True,True
16185,R241,28.857,Baily with Steel Deck,LRP028c,MAJIDPUR TRUSS BRIDGE,37.9,B,114992,Auskandi-Raniganj-Jagannathpur-Pagla Road,28.857,...,Sylhet,Sylhet,Sunamganj,Chattak,91.542083,24.796694,bcs1,False,True,True
17247,R241,28.211,Box Culvert,LRP028a,HIJLA BOX CULVERT,9.2,C,114993,Auskandi-Raniganj-Jagannathpur-Pagla Road,28.211,...,Sylhet,Sylhet,Sunamganj,Chattak,91.541778,24.790944,bcs1,False,True,True
17248,R241,32.128,Box Culvert,LRP032a,KHASIRA BOX CULVERT,6.0,C,114989,Auskandi-Raniganj-Jagannathpur-Pagla Road,32.128,...,Sylhet,Sylhet,Sunamganj,Chattak,91.534361,24.823833,bcs1,False,True,True


# The code above takes a lot of time to run (45mins), so I made a new code below that SHOULD run faster, but turns out that this runs slower (47mins) :))))

In [ ]:
# Create geometries using vectorized operations with GeoDataFrame
valid_mask = df_bridge['lat'].notna() & df_bridge['lon'].notna()

# Vectorized point creation using GeoDataFrame
gdf_original = gpd.GeoDataFrame(
    df_bridge[valid_mask],
    geometry=gpd.points_from_xy(df_bridge.loc[valid_mask, 'lon'], df_bridge.loc[valid_mask, 'lat']),
    crs=bgd.crs
)

gdf_swapped = gpd.GeoDataFrame(
    df_bridge[valid_mask],
    geometry=gpd.points_from_xy(df_bridge.loc[valid_mask, 'lat'], df_bridge.loc[valid_mask, 'lon']),
    crs=bgd.crs
)

# Use spatial join / within check vectorized via GeoSeries
df_bridge['in_boundary'] = False
df_bridge['in_boundary_swapped'] = False

df_bridge.loc[valid_mask, 'in_boundary'] = gdf_original.within(boundary).values
df_bridge.loc[valid_mask, 'in_boundary_swapped'] = gdf_swapped.within(boundary).values

df_bridge['lat_lon_swapped_flag'] = (~df_bridge['in_boundary']) & (df_bridge['in_boundary_swapped'])

# Show flagged rows
df_bridge[df_bridge['lat_lon_swapped_flag']]

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,...,zone,circle,division,sub-division,lat,lon,EstimatedLoc,in_boundary,in_boundary_swapped,lat_lon_swapped_flag
2864,R241,29.840,Box Culvert,LRP029a,MAJIDEPUR BOX CULVERT,3.0,A,114991,Auskandi-Raniganj-Jagannathpur-Pagla Road,29.84,...,Sylhet,Sylhet,Sunamganj,Chattak,91.543889,24.805111,bcs1,False,True,True
2865,R241,31.212,Baily with Steel Deck,LRP031a,KHASIRA BAILEY WITH STEEL DECK,17.5,A,114990,Auskandi-Raniganj-Jagannathpur-Pagla Road,31.212,...,Sylhet,Sylhet,Sunamganj,Chattak,91.538722,24.816556,bcs1,False,True,True
2866,R241,32.713,Box Culvert,LRP032b,KOLKALI BOX CULVERT,3.0,A,114988,Auskandi-Raniganj-Jagannathpur-Pagla Road,32.713,...,Sylhet,Sylhet,Sunamganj,Chattak,91.530556,24.828056,bcs1,False,True,True
2867,R241,33.126,Box Culvert,LRP033a,KOKAL BOX CULVERT,1.0,A,114987,Auskandi-Raniganj-Jagannathpur-Pagla Road,33.126,...,Sylhet,Sylhet,Sunamganj,Chattak,91.527917,24.830944,bcs1,False,True,True
2868,R241,34.978,Box Culvert,LRP035a,TARAKHAI BOX CULVERT,14.4,A,114984,Auskandi-Raniganj-Jagannathpur-Pagla Road,34.978,...,Sylhet,Sylhet,Sunamganj,Chattak,91.518194,24.844972,bcs1,False,True,True
2869,R241,36.164,Truss with Steel Deck,LRP036a,COUNDER NOLA BRIDGE,40.6,A,114982,Auskandi-Raniganj-Jagannathpur-Pagla Road,36.164,...,Sylhet,Sylhet,Sunamganj,Chattak,91.514083,24.855083,bcs1,False,True,True
16183,R241,26.292,Box Culvert,LRP026a,EKOR CHOL BOX CULVERT,12.0,B,114995,Auskandi-Raniganj-Jagannathpur-Pagla Road,26.292,...,Sylhet,Sylhet,Sunamganj,Chattak,91.544194,24.773694,bcs1,False,True,True
16185,R241,28.857,Baily with Steel Deck,LRP028c,MAJIDPUR TRUSS BRIDGE,37.9,B,114992,Auskandi-Raniganj-Jagannathpur-Pagla Road,28.857,...,Sylhet,Sylhet,Sunamganj,Chattak,91.542083,24.796694,bcs1,False,True,True
17247,R241,28.211,Box Culvert,LRP028a,HIJLA BOX CULVERT,9.2,C,114993,Auskandi-Raniganj-Jagannathpur-Pagla Road,28.211,...,Sylhet,Sylhet,Sunamganj,Chattak,91.541778,24.790944,bcs1,False,True,True
17248,R241,32.128,Box Culvert,LRP032a,KHASIRA BOX CULVERT,6.0,C,114989,Auskandi-Raniganj-Jagannathpur-Pagla Road,32.128,...,Sylhet,Sylhet,Sunamganj,Chattak,91.534361,24.823833,bcs1,False,True,True


# LRPs that out of the bangladesh polygon including the Nan in lon or lat, and swapped coordinates

In [8]:
# count row with in_boundary False
count_outside = df_bridge[~df_bridge['in_boundary']].shape[0]
print(f"Number of rows with in_boundary False: {count_outside}")

Number of rows with in_boundary False: 231


In [10]:
# display rows with in_boundary False
outside_rows = df_bridge[~df_bridge['in_boundary']]
outside_rows.head()

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,...,zone,circle,division,sub-division,lat,lon,EstimatedLoc,in_boundary,in_boundary_swapped,lat_lon_swapped_flag
855,N2,246.910,Box Culvert,LRP247a,KHAN BAGAN,6.5,A,103778,Dhaka (Katchpur)-Bhairab-Jagadishpur-Shaistaga...,246.91,...,Sylhet,Sylhet,Sylhet,Sylhet,25.963389,92.019083,bcs1,False,False,False
1932,N104,17.410,RCC Bridge,LRP017b,RAMNAGAR BRIGE,6.0,A,101821,Feni-Noakhali Road,17.41,...,Comilla,Noakhali,Feni,Feni-1,23.941194,91.276472,bcs1,False,False,False
2083,N108,14.159,Box Culvert,LRP014b,LAMBA RASTA (2),1.5,A,101753,Keranirhat-Bandarban Road,14.159,...,Chittagong,Rangamati,Bandarban,Bandarban-2,20.155611,92.178889,bcs1,False,False,False
2280,R151,5.288,Box Culvert,LRP005d,SULTURBA CULVERT,3.1,A,104888,Baraiyerhat-Karerhat-Heako-Narayanhat-Fatikchh...,5.288,...,Chittagong,Chittagong,Chittagong,Sitakunda,21.924028,91.558611,bcs1,False,False,False
2469,R164,0.116,Box Culvert,LRPSa,BARAICHARI BOX CULVERT,6.0,A,106763,Baraichhari-Kaptai Road,0.116,...,Chittagong,Chittagong,Chittagong,Chittagong,22.503194,92.640028,bcs1,False,False,False


In [15]:
# display rows with inbpoundary False and lon or lat values is nan or zero
outside_rows_nan_zero = outside_rows[(outside_rows['lon'].isna()) | (outside_rows['lat'].isna()) | (outside_rows['lon'] == 0) | (outside_rows['lat'] == 0)]
#print(outside_rows_nan_zero.head())
print(f"Number of rows with in_boundary False and lon or lat values is nan or zero: {outside_rows_nan_zero.shape[0]}")

Number of rows with in_boundary False and lon or lat values is nan or zero: 111


In [17]:
# print rows with in_boundary false and in_boundary_swapped true
swapped_rows = df_bridge[df_bridge['lat_lon_swapped_flag']]
print(f"Number of rows with in_boundary False and in_boundary_swapped True: {swapped_rows.shape[0]}")

Number of rows with in_boundary False and in_boundary_swapped True: 15
